In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing libraries
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("/kaggle/input/anz-synthesised-transaction-dataset/anz.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.bpay_biller_code.value_counts()

In [ ]:
df.drop(['bpay_biller_code','merchant_code'],axis=1,inplace=True)

In [ ]:
df.isna().sum()

Here, we find exact number of null values for merchant data

let's check if the missing values are common for rows

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df)

we can find the missing values are common with rows

let's confirm by selecting only null based columns and compare pattern

In [ ]:
msno.matrix(df[['card_present_flag','merchant_id','merchant_suburb','merchant_state','merchant_long_lat']])

we can now confirm that missing values are at common rows. 
let's eliminate the rows

In [ ]:
df.dropna(inplace=True)

Let's recheck the whole dataframe

In [ ]:
msno.matrix(df)

In [ ]:
print(f"we are left with {df.shape[0]} rows and {df.shape[1]} columns")

let's check for each column

# Status

In [ ]:
df.status.value_counts()

In [ ]:
plt.plot(df.status)

let's remove this as it has only one value and will create noise in predictive analysis

In [ ]:
df.drop(['status'],axis=1,inplace=True)

# card_present_flag

In [ ]:
df['card_present_flag'].value_counts()

In [ ]:
plt.hist(df['card_present_flag'])

card_present_flag is pretty much imbalanced

# account

In [ ]:
df['account'].value_counts()

these are probably the account numbers and won't much add value to the predictions

let's drop this column too

In [ ]:
df.drop(['account'],axis=1,inplace=True)

# currency

In [ ]:
df['currency'].value_counts()

Dropping this column too because of single value nature

In [ ]:
df.drop(['currency'],axis=1,inplace=True)

# long_lat

In [ ]:
df.long_lat.value_counts()

1. lets visualise these co-ordinates
2. seperate lat and long for visualisation
3. convert str to float
4. plot on basemap

In [ ]:
# new data frame with split value columns 
co_ordinates = df["long_lat"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
df["longitude"]= co_ordinates[0] 
  
# making separate last name column from new data frame 
df["latitude"]= co_ordinates[1] 

In [ ]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [ ]:
from mpl_toolkits.basemap import Basemap
fig = plt.figure(figsize=(12,9))
m = Basemap(projection='mill',
           llcrnrlat = -90,
           urcrnrlat = 90,
           llcrnrlon = -180,
           urcrnrlon = 180,
           resolution = 'c')
m.drawcoastlines()
m.drawparallels(np.arange(-90,90,10),labels=[True,False,False,False])
m.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])
sites_lat_y = df['latitude'].tolist()
sites_lon_x = df['longitude'].tolist()
m.scatter(sites_lon_x,sites_lat_y,latlon=True)
plt.title('Basemap', fontsize=20)
plt.show()

In [ ]:
df.drop(['long_lat'],axis=1,inplace=True)

# txn_description

In [ ]:
df['txn_description'].value_counts()

In [ ]:
plt.hist(df['txn_description'])

1. the column data is balanced and evenly distributed
2. let's convert this strings into numeric

In [ ]:
cleanup_txn = {"txn_description":{"POS": 1, "SALES-POS":0}}
df.replace(cleanup_txn, inplace=True)

# merchant_id

In [ ]:
df['merchant_id'].value_counts()

this data is just rubbish and of no use.

In [ ]:
df.drop(['merchant_id'],axis=1,inplace=True)

# first_name

In [ ]:
df.first_name.value_counts()

# balance

In [ ]:
df.balance.value_counts()

In [ ]:
plt.hist(df.balance)

we can see some outliers here

In [ ]:
df=df[df['balance']<100000]

In [ ]:
plt.hist(df.balance)

# date 

In [ ]:
df.date.value_counts()

In [ ]:
type(df.date[0])

converting string to date type 

splitting date into only month and year

In [ ]:

df['date']= pd.to_datetime(df['date']) 
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
df.year.value_counts()

since we have all the transactions in year 2018, it's obvious to remove this column

In [ ]:
df.drop(['date','year'],axis=1,inplace=True)

In [ ]:
df.month.value_counts()

we can see only that transactions happened only during august, september and october

# gender

In [ ]:
df['gender'].value_counts()

good proportion.

let's convert this into numeric form

In [ ]:
cleanup_gender = {"gender":{"M": 1, "F":0}}
df.replace(cleanup_gender, inplace=True)

# age

In [ ]:
plt.hist(df.age)

there are some outlier like values but they can have some meaning. 

let's not remove them.

# merchant_suburb

In [ ]:
df['merchant_suburb'].nunique()

encoding such unique values will increase shape and noise.

let's not make any chances to this column

# merchant_state

In [ ]:
df['merchant_state'].value_counts()

In [ ]:
df.extraction.value_counts()

these values are probably the timestamps

let's remove for now.

In [ ]:
df.drop(['extraction'],axis=1,inplace=True)

# amount

In [ ]:
plt.hist(df.amount)

In [ ]:
df.amount.max()

this seems as an outlier

In [ ]:
perc =[.80,.90,.99] 
df.amount.describe(percentiles=perc)

In [ ]:
df=df[df['amount']<378]

# transaction_id

In [ ]:
df['transaction_id'].value_counts()

In [ ]:
df.drop(['transaction_id'],axis=1,inplace=True)

# country

In [ ]:
df['country'].value_counts()

In [ ]:
df.drop(['country'],axis=1,inplace=True)

# customer_id

In [ ]:
df['customer_id'].value_counts()

In [ ]:
df.drop(['customer_id'],axis=1,inplace=True)

In [ ]:
df.columns

# merchant_long_lat

In [ ]:
df['merchant_long_lat'].value_counts()

In [ ]:
df.drop(['merchant_long_lat'],axis=1,inplace=True)

# movement

In [ ]:
df['movement'].value_counts()

single class. drop

In [ ]:
df.drop(['movement'],axis=1,inplace=True)

# final data

In [ ]:
df